In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
%matplotlib inline
import argparse
import os
import pprint
import shutil
import pickle
import cv2
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import random

import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
#from tensorboardX import SummaryWriter

import _init_paths
from config import cfg
from config import update_config
from core.loss import JointsMSELoss
from core.function import train
from core.function import validate
from utils.utils import get_optimizer
from utils.utils import save_checkpoint
from utils.utils import create_logger
from utils.utils import get_model_summary

import dataset
import models

In [7]:
with open('/u/snaha/v6/dataset/AWA/Animals_with_Attributes2/quadruped_keypoints/coco_format/dataset.pickle', 'rb') as handle:
    dataset = pickle.load(handle)
    
dataset.keys()

dict_keys(['annotations', 'images', 'categories', 'info'])

In [8]:
dataset['images'][0]

{'width': 1024,
 'height': 768,
 'filename': 'antelope_10002.jpg',
 'id': 'antelope_10002'}

In [9]:
path = '/u/snaha/v6/dataset/AWA/Animals_with_Attributes2/quadruped_keypoints/Annotations/'
list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

animal_list = []

for animal in list_subfolders_with_paths:
    animal_folder = animal#os.path.join(path, animal)
    animal_list.append(animal_folder.split('/')[-1])
    #import ipdb; ipdb.set_trace()
    #exit(0)
    
animal_list    

> <ipython-input-9-7ed0bd7e3e73>(10)<module>()
      9     import ipdb; ipdb.set_trace()
---> 10     exit(0)
     11 

ipdb> animal_folder
'/u/snaha/v6/dataset/AWA/Animals_with_Attributes2/quadruped_keypoints/Annotations/antelope'
ipdb> animal_list
['antelope']
ipdb> exit(0)


BdbQuit: 

In [5]:
def single_animal_split(single_animal_img_list, full_body_im_list, test_im_per_animal = 20):

    train_anno_list = []
    train_img_list = []
    
    test_anno_list = []
    test_img_list = []
        
    rand_inds = np.arange(len(full_body_im_list)).tolist()
    random.shuffle(rand_inds)
    
    test_img_name_list = []
    
    rand_inds = rand_inds[:test_im_per_animal]
    
    for i in rand_inds:
        test_img_name_list.append(full_body_im_list[i])
        
    for im, anno in single_animal_img_list:
        if im['filename'].split('.')[0] in test_img_name_list:
            test_anno_list.append(anno)
            test_img_list.append(im)
        else:
            train_anno_list.append(anno)
            train_img_list.append(im)            
            
        #import ipdb; ipdb.set_trace()
        #exit(0)
    
    return train_anno_list, train_img_list, test_anno_list, test_img_list

def generate_split(animal_list, dataset, all_full_animal_image_list_dict):
    
    dataset_train = dict()
    dataset_test = dict() 
    
    annotations_train = []
    annotations_test = []
    
    images_train = []
    images_test = []
    
    animal_dict = defaultdict(list)
    
    for im, anno in zip(dataset['images'], dataset['annotations']):
        #import ipdb; ipdb.set_trace()
        #exit(0)     
        animal = anno['animal']
        animal_dict[animal].append( (im, anno) )
                
    for animal in animal_dict:
        train_anno_list, train_img_list, test_anno_list, test_img_list = single_animal_split(animal_dict[animal], all_full_animal_image_list_dict[animal])
            
        annotations_test.extend(test_anno_list)
        images_test.extend(test_img_list)
        
        annotations_train.extend(train_anno_list)
        images_train.extend(train_img_list)
        #import ipdb; ipdb.set_trace()
        #exit(0)  
            
            
    dataset_train['annotations'] = annotations_train
    dataset_test['annotations'] = annotations_test
    
    dataset_train['images'] = images_train
    dataset_test['images'] = images_test
    
    dataset_train['categories'] = dataset['categories']
    dataset_test['categories'] = dataset['categories']    
    
    dataset_train['info'] = dataset['info']
    dataset_test['info'] = dataset['info']       
    
    return dataset_train, dataset_test

#all_full_animal_image_list_dict = 
with open('/u/snaha/v6/dataset/AWA/Animals_with_Attributes2/quadruped_keypoints/Annotations/all_full_animal_image_list_dict.pickle', 'rb') as handle:
    all_full_animal_image_list_dict = pickle.load(handle)
    
for i in range(5):
    dataset_train, dataset_test = generate_split(animal_list, dataset, all_full_animal_image_list_dict)
    #test_animals
    print('Number of train instances = ', len(dataset_train['images']), len(dataset_train['annotations']))
    print('Number of test instances = ', len(dataset_test['images']), len(dataset_test['annotations']))
    
    with open('/u/snaha/v6/dataset/AWA/Animals_with_Attributes2/quadruped_keypoints/coco_format/supervised_dataset_train_' + str(i+1) + '.pickle', 'wb') as handle:
        pickle.dump(dataset_train, handle, protocol=pickle.HIGHEST_PROTOCOL)    
        
    with open('/u/snaha/v6/dataset/AWA/Animals_with_Attributes2/quadruped_keypoints/coco_format/supervised_dataset_val_' + str(i+1) + '.pickle', 'wb') as handle:
        pickle.dump(dataset_test, handle, protocol=pickle.HIGHEST_PROTOCOL)            

Number of train instances =  9405 9405
Number of test instances =  647 647
Number of train instances =  9399 9399
Number of test instances =  653 653
Number of train instances =  9400 9400
Number of test instances =  652 652
Number of train instances =  9400 9400
Number of test instances =  652 652
Number of train instances =  9392 9392
Number of test instances =  660 660


In [12]:
a = [.24, .72, .72, .18, .62, .23, .56, .26, .38, .28, .29, .48, .23, .60, .47, .24, .72, .72, .18, .62, .23, .56, .26, .38, .28, .29, .48, .23, .60, .47, .18, .62, .23, .56, .26, .38, .28, .29, .48]
len(a)

39